for video

In [41]:
import cv2
import numpy as np
import time
import math
import matplotlib.pylab as plt
import random
from sklearn.metrics import mean_squared_error 
 
class Profiler(object):
    def __enter__(self):
        self._startTime = time.time()
         
    def __exit__(self, type, value, traceback):
        print( "Elapsed time: {:.3f} sec".format(time.time() - self._startTime))

from timeit import default_timer as timer

def definition_of_new_position(current_position, time_shooting, angular_velocity, centr):
    angular = angular_velocity * time_shooting
    x = math.fabs(current_position[0] - centr[0])*math.cos(angular) + math.fabs(current_position[1] - centr[1])*math.sin(angular)
    y = - math.fabs(current_position[0] - centr[0])*math.sin(angular) + math.fabs(current_position[1] - centr[1])*math.cos(angular)
    if current_position[0] < centr[0]:
        x = - x
    if current_position[1] < centr[1]:
        y = -y 
    return (centr[0]+x),(centr[1] + y)

def definition_of_centr(x1,y1,x2,y2,x3,y3):
    A = x2 - x1
    B = y2 - y1
    C = x3 - x1
    D = y3 - y1
    E = A * (x1 + x2) + B * (y1 + y2)
    F = C * (x1 + x3) + D * (y1 + y3)
    G = 2 * (A * (y3 - y2) - B * (x3 - x2))
    if G!=0:
        Cx = (D * E - B * F) / G
        Cy = (A * F - C * E) / G
        R = math.sqrt((Cx-x1)**2+(Cy-y1)**2)
        return Cx, Cy, R

def scalarh(x_1,y_1,x_2,y_2,x_3,y_3):
    x_2 = math.fabs(x_1-x_2)
    x_3 = math.fabs(x_1-x_3)
    y_2 = math.fabs(y_1-y_2)
    y_3 = math.fabs(y_1-y_3)
    sm = (x_2*x_3+y_2*y_3)
    f = (math.sqrt(x_2**2+y_2**2)* math.sqrt(x_3**2+y_3**2))
    return math.acos(sm/f) 

def superimposition_mask(x,y):
    new = np.zeros(y.shape)
    for i in range(y.shape[2]):
        new[:,:,i] = x*y[:,:,i]
    return new

def mask(a, x1, x2, y1, y2, z1, z2):
    h_min = np.array((x1, y1, z1), np.uint8)
    h_max = np.array((x2, y2, z2), np.uint8)
    return cv2.inRange(a, h_min, h_max)

def im_cir(img,f,final):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )
    thresh = mask(hsv, 0,238,166,255,157,255)
    new = superimposition_mask(thresh, img)
    #print(new[0][0])
    new = new.astype(np.uint8)
    #print(new[0][0])
    new = cv2.medianBlur(new,5)
    new = cv2.cvtColor(new, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite('5.png',new)
    new = cv2.Canny(new, 350, 360)
    #cv2.imwrite('6.png',new)
    #hsv = cv2.cvtColor(new, cv2.COLOR_HSV2BGR )
    #hsv = cv2.cvtColor(hsv, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(new, cv2.HOUGH_GRADIENT, 1.15,0.0001,param1=85,param2=100,minRadius=5,maxRadius=300)
    print(circles.shape)
    try:
        #circles = np.uint16(np.around(circles))
        circles = circles.reshape(circles[0].shape) 
        circles_mean = np.mean(circles, axis = 0)
        circles_mean  = np.uint16(np.around(circles_mean))
        #for i in circles[0,:]:
    # нарисовать центры окружностей
           # cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
            #cv2.circle(img,(i[0],i[1]),2,(0,0,255),3)
        cv2.circle(img,(circles_mean[0],circles_mean[1]),2,(0,0,255),3)
    #cv2.imshow('result', new) 
        final[f,[0,1,2]] =  circles_mean
        cv2.imwrite("videos/new"+str(f)+".jpg",img)
    except Exception as e:
        print(e)

In [42]:
cap = cv2.VideoCapture("result_720.mp4")
s=0
l = False
speed = np.zeros(shape=(300))
final = np.zeros(shape=(300,4))
while (s < 300):    
    #with Profiler() as p:
    flag, img = cap.read()
    timef = cap.get( cv2.CAP_PROP_POS_MSEC )
    if flag:
        im_cir(img,s,final)
        #print(flag)
        #print(s)
    final[s,3]= timef
    print(timef)
    if l>1:
        speed[s] = scalarh(x0,y0,x_new,y_new,final[s,0],final[s,1] ) / ((final[s,3] - final[s-1,3])/1000)
    x_new = final[s,0]    
    y_new = final[s,1]
    l +=1
    s+=1
   
    

(1, 1, 3)
40.018893387314435
(1, 5, 3)
80.03778677462887
(1, 6, 3)
120.05668016194332
(1, 10, 3)
160.07557354925774
(1, 4, 3)
200.0944669365722
(1, 4, 3)
240.11336032388664
(1, 7, 3)
280.1322537112011
(1, 5, 3)
320.1511470985155
(1, 4, 3)
360.17004048582993
(1, 1, 3)
400.1889338731444
(1, 1, 3)
440.2078272604588
(1, 2, 3)
480.2267206477733
(1, 4, 3)
520.2456140350877
(1, 5, 3)
560.2645074224022
(1, 5, 3)
600.2834008097166
(1, 6, 3)
640.302294197031
(1, 5, 3)
680.3211875843455
(1, 7, 3)
720.3400809716599
(1, 11, 3)
760.3589743589744
(1, 8, 3)
800.3778677462888
(1, 10, 3)
840.3967611336031
(1, 8, 3)
880.4156545209177
(1, 8, 3)
920.434547908232
(1, 8, 3)
960.4534412955466
(1, 9, 3)
1000.472334682861
(1, 8, 3)
1040.4912280701753
(1, 16, 3)
1080.5101214574897
(1, 9, 3)
1120.5290148448044
(1, 8, 3)
1160.5479082321187
(1, 4, 3)
1200.5668016194331
(1, 7, 3)
1240.5856950067475
(1, 11, 3)
1280.604588394062
(1, 9, 3)
1320.6234817813765
(1, 3, 3)
1360.642375168691
(1, 8, 3)
1400.6612685560053
(1, 

(1, 8, 3)
11485.422402159244
(1, 8, 3)
11525.441295546558
(1, 10, 3)
11565.460188933872
(1, 11, 3)
11605.479082321188
(1, 8, 3)
11645.497975708502
(1, 10, 3)
11685.516869095816
(1, 7, 3)
11725.53576248313
(1, 10, 3)
11765.554655870445
(1, 10, 3)
11805.573549257759
(1, 9, 3)
11845.592442645073
(1, 10, 3)
11885.611336032389
(1, 6, 3)
11925.630229419703
(1, 8, 3)
11965.649122807017
(1, 10, 3)
12005.668016194331


In [ ]:
definition_of_new position((570,549),0.040,np.mean(speed),(x0,y0))

In [ ]:
np.mean(speed)

In [ ]:
final[0]

In [ ]:
scalarh(x0,y0,684,551,678,559)

In [ ]:
final = np.uint16(np.around(final))
for i in final[0:]:
    cv2.circle(img,(i[0],i[1]),2,(0,0,255),3)
cv2.imwrite('Final.png',img)


In [ ]:
bag = cv2.imread('nn.jpg',0)
bag = cv2.cvtColor(bag,cv2.COLOR_GRAY2BGR)

final = np.uint16(np.around(final))
for i in final[0:]:
    cv2.circle(bag,(i[0],i[1]),3,(255,255,255),3)
cv2.imwrite('Final_for_bag.png',bag)

In [44]:
#Получить изображение из файла
img = cv2.imread('Final_for_bag.png',0)
#Фильтрация изображения
img = cv2.medianBlur(img,5)
img = cv2.medianBlur(img,5)
img = cv2.GaussianBlur(img,(11,11),10)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,2,0.001,param1=40,param2=170,minRadius=1,maxRadius=500)
b = 0
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # нарисовать окружности
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # нарисовать центры окружностей
    cv2.circle(cimg,(i[0],i[1]),2,(0,255,0),3)
    b +=1

cv2.imwrite('Circles.png',cimg)
print(circles)
circles = circles.reshape(circles[0].shape) 
circles_mean = np.mean(circles, axis = 0)

[[[647 381 182]
  [645 365 186]
  [643 383 186]
  [645 371 180]
  [643 363 183]
  [643 375 178]
  [631 389 194]
  [621 381 190]
  [653 385 193]
  [627 385 180]
  [659 387 171]]]


In [43]:
r = 184
x0= 642
y0 = 379

In [22]:
from operator import itemgetter

In [73]:
k = 0
er = 0
h = 0
radius1 = np.zeros(shape=(300))
radius2 = np.zeros(shape=(300))
rez = np.zeros (shape=(300,3))
mse = np.zeros(shape=(300))
while (k < 300):
    s1 = random.randint(1, s-3) 
    s2 = random.randint(s1-2, s-2) 
    s3 = random.randint(s2-2, s-1)
    try: 
        rez[h] = definition_of_centr(final[s1,0],final[s1,1],final[s2,0],final[s2,1],final[s3,0],final[s3,1])   
        for i in final:
            radius1[h] = (i[0] -rez[h,0])**2+(i[1] - rez[h,1])**2
            radius2[h] = r
        mse[h] = mean_squared_error(radius1,radius2)
        k +=1
        h+=1
    except:
        print("G = 0")



G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0
G = 0


In [77]:
np.argmin(mse)

0

In [75]:
np.min(mse)

5027997.977464453

In [4]:
def definition_of_new_position(current_position, time_shooting, angular_velocity, centr):
    angular = angular_velocity * time_shooting
    x = math.fabs(current_position[0] - centr[0])*math.cos(angular) + math.fabs(current_position[1] - centr[1])*math.sin(angular)
    y = - math.fabs(current_position[0] - centr[0])*math.sin(angular) + math.fabs(current_position[1] - centr[1])*math.cos(angular)
    if current_position[0] < centr[0]:
        x = - x
    if current_position[1] < centr[1]:
        y = -y 
    return (centr[0]+x),(centr[1] + y)
